In [ ]:
# We will start by fixing the seed for reproducibility
seed = 0
import torch
torch.manual_seed(seed)
import numpy as np
np.random.seed(seed)
import random
random.seed(seed)

In [ ]:
try:
    import pgmuvi
except ImportError:
    %pip install git+https://github.com/ICSM/pgmuvi.git

In [ ]:
#now generate some data from a more complex case, maybe drawing from a specific PSD to test if pgmuvi reconstructs it
#timestamps_2d = #generate random x data here
#wavelengths_2d = #generate random x data here
#fluxes_2d = #generate random y data here
#flux_err_2d =
P0, P1 = 1000, 220
n_data = 50
n_data1 = n_data
n_data2 = n_data // 2
jd_min, jd_max = 2450000, 2455000
timestamps1_2d = torch.Tensor(np.random.uniform(jd_min, jd_max, size=n_data1))
timestamps2_2d = torch.Tensor(np.random.uniform(jd_min, jd_max, size=n_data2))
timestamps_2d = torch.concat((timestamps1_2d, timestamps2_2d), dim=0)

bands1_2d = torch.Tensor(np.array([0.854] * n_data1))
bands2_2d = torch.Tensor(np.array([2.206] * n_data2))
bands_2d = torch.concat((bands1_2d, bands2_2d), dim=0)

flux1_2d = torch.Tensor(np.sin(timestamps1_2d * (2 * np.pi / P0)))
flux1_2d += 0.1*torch.randn_like(flux1_2d)
flux1_err_2d = 0.1*flux1_2d

flux2_2d = torch.Tensor(np.sin(timestamps2_2d * (2 * np.pi / P1) + 0.255 * jd_min))
flux2_2d += 0.18*torch.randn_like(flux2_2d)
flux2_err_2d = 0.02*flux2_2d

fluxes_2d = torch.concat((flux1_2d, flux2_2d), dim=0)
flux_err_2d = torch.concat((flux1_err_2d, flux2_err_2d), dim=0)

print(timestamps_2d.shape, bands_2d.shape, fluxes_2d.shape, flux_err_2d.shape)

timestamps_bands_2d = torch.stack((timestamps_2d, bands_2d), 0).T
timestamps_bands_2d.shape

In [ ]:
from pgmuvi.lightcurve import Lightcurve

lightcurve_2d = Lightcurve(timestamps_bands_2d, fluxes_2d, yerr=flux_err_2d, xtransform='minmax')

In [ ]:
lightcurve_2d._xdata_transformed

In [ ]:
fit_2d = lightcurve_2d.fit(model='2D', likelihood='learn')

In [ ]:
fit_2d['covar_module.mixture_means'][-1][...,1]

In [ ]:
lightcurve_2d.print_results()